Notebook copied from Alla test notebook after running all the experiments and sending to the professor the report. The ideal will be to modify this file to get the final notebook to send to the prof.

# Packages

In [1]:
from itertools import groupby # used when finding the partition of a given matrix A
import numpy as np
import random # used to generate de partition
from scipy.stats import random_correlation # to generate the correlation matrix which is positive semidefinite
from scipy.sparse import csr_matrix, csc_matrix, eye, vstack # types of sparse matrices, and to stack vertic. matrices
from scipy.linalg import cho_factor, cho_solve, null_space # cholesky factorization
from tabulate import tabulate
import time

SEED = 13
random.seed(SEED) 
np.random.seed(SEED) 

# Functions to generate the quadratic problems

In [2]:
def generate_symmetric_psd_matrix(n,rank):

    assert rank <= n, "Rank k must be less than to n"

    # Random matrices for rank-k construction
    A = np.random.randn(n, rank)  # m x k matrix
    B = np.random.randn(rank, n)  # k x m matrix

    # Resultant matrix
    Q = np.dot(A, B)

    # Construct psd matrix by multiplying QQ^T
    Q = np.dot(Q,Q.T)
    return Q

   
def generate_symmetric_pd_matrix(n):
    '''Function that returns a symmetric positive definite matrix.'''
    
    def generate_full_rank_matrix(n):
        while True:
            G = np.random.rand(n, n)  # Generate random matrix of size n x n
            if np.linalg.matrix_rank(G) == n:  # Check if it's full rank
                return G  
            
    G = generate_full_rank_matrix(n)
    G_T = np.transpose(G)
    Q = np.dot(G_T, G)
    return Q

def define_partition(n, k):
    '''Function that returns a random partition of size k of the set {1...n}'''
    
    lst = list(range(0,n))
    random.shuffle(lst) # shuffle the list to compose sets with different elements
    indexes = sorted(random.sample(range(0,n-1), k - 1)) # up to n-1 bc we don't want to take as index the last one to don't generate empty subsets
    cand_partition = [np.array(lst[start:end]) for start, end in zip([0] + indexes, indexes + [len(lst)])]
    return cand_partition

def partition(n,k):
    while True:
        partition = define_partition(n,k)
        if all(len(subs) > 0 for subs in partition):
            return partition
        
def matrix_of_constraints(partition): 

    '''Function that returns the matrix of constraints (in sparse format) defined by the given partition.'''
    
    n = sum([len(subset) for subset in partition])
    partition = [subset for subset in partition]

    # extract the indexes of the positions with ones
    cols_idx = np.concatenate(partition)
    rows_idx = [idx for idx, set in enumerate(partition) for _ in range(len(set))]

    # create the matrix A
    A = np.zeros((len(partition), n))
    A[rows_idx, cols_idx] = 1

    A = csc_matrix(A)
    return A

def define_vector_b(k):
  'Generates a vector of ones of length k'
  b = np.ones((k,1))
  return b

# Generate the initial vector
def initial_point(partition):
    '''Generates the vector x0 of size n such that the i-th position is 1 
    if i is the smallest element in one of the subsets of the partition; 
    otherwise, it is 0.'''

    n = sum([len(subset) for subset in partition])
    positions_ones = [min(sub) for sub in partition]
    
    x0 = np.zeros((n,1))
    np.add.at(x0, positions_ones, 1) # increment items of x0 in position 'positions' by 1.
    # L = [i for i in range(n) if i not in positions_ones]
    return x0

def generate_c_vector(n):
    '''Generate the vector representing the linear part of the objective function'''
    c = np.random.rand(n)
    c = c.reshape((n,1))
    return c

def generate_special_c_vector(Q,x_0):
    c = np.matmul(Q,x_0)*-(1)
    return c

# Intermediate functions

In [4]:
def find_the_partition(A):
    '''Returns the partition associataed to the given matrix A of constraints'''
    
    positions = np.argwhere(A==1)
    
    #to use groupby the iterable needs to be sorted on the key function. np.argwhere returns it sorted
    partition = [[col for row, col in group] for idxset, group in groupby(positions, key=lambda x: x[0])]
    
    return partition

In [5]:
def find_permutation_matrix(U, partition):
    """ Find the permutation matrix P given a partition and a set U of column indexes such that 
        AP = [I|Az], where A is a matrix of constraints"""    
    els_U= [list(set(Pi).intersection(U)) for Pi in partition]

    cumul_len = 0
    cols_idxs = []
    k = len(els_U)

    for i, subs in enumerate(els_U):
        
        cols_idxs.append(i)
        n_dependent_cols = len(subs)-1
        dep_pos = [k+cumul_len+j for j in range(n_dependent_cols)]
        cols_idxs.extend(dep_pos)
        cumul_len = cumul_len + n_dependent_cols

    U_flatten = [idx for subs in els_U for idx in subs]
    U_flatten_copy = U_flatten.copy()
    U_flatten.sort()
    
    new_positions = [(el_u, i) for i, el_u in enumerate(U_flatten)]
    ordered_new_positions = sorted(new_positions, key=lambda x: U_flatten_copy.index(x[0]))
    
    rows_idxs = [x[1] for x in ordered_new_positions]

    data = np.ones(len(rows_idxs))

    permutation_matrix = csc_matrix((data, (rows_idxs, cols_idxs)), shape=(len(rows_idxs),len(rows_idxs)))

    return permutation_matrix

In [6]:
# Az is of size kx(u-k)
def compute_Z(AuPerm, k, u):
    ''' Returns the matrix Z = [-A_z] used to find the direction p
                               [  I ] '''  
    AZ = AuPerm[:, k:]
    id_Z = eye(u-k)
    Z = vstack((-AZ, id_Z))
    return Z

In [7]:
def compute_pz(Z, perm_m, Qu, Y, py, g_u):
    '''pz is the vector obtained by solving the equation system
        (PZ)^TQ_U(PZ)pz = -(PZ)^T(QuPYpy+gu)
        where P=perm_m'''
    
    if Z.size == 0:
        pz = np.empty(0)
        return 1, pz
    else:
        # Find (PZ)^TQ_u(PZ),
        PZ = perm_m@Z
        PZTQ = PZ.transpose()@Qu
        PZQPZ = PZTQ@PZ.toarray()
        
        try:
            # Z^TQ_UYpy
            PY = perm_m@Y
            QPY = Qu@PY
            QPYPy = QPY@py

            term_r = -QPYPy - g_u
            term_r = PZ.transpose()@term_r
            term_r = term_r

            # Find the Cholesky decomposition of (PZ)^TQPZ.
            # If the matrix is singular we get an error.
            ch, low = cho_factor(PZQPZ) 

            # Solve the linear equation (PZ)^TQPZ pz = term_r
            pz = cho_solve((ch, low), term_r)
            return 1, pz
        except Exception as e:
            # print(f"Cholesky factorization failed: {e}")
            return 0, [] 

In [8]:
def compute_lambdas(perm_m, Y, g_u, Qu, p_u):
    '''Compute the Lagrange multipliers of the equality constraints as (PY)^T(gu+Qupu)'''
    
    PY = perm_m@Y
    Qup_u = Qu@p_u
    r_term = g_u+Qup_u

    lambdas = PY.T@r_term
    return lambdas

def compute_mult_ineq(Q, x, c, mult_eq, A):
    '''Compute the Lagrange multipliers of the inequality constraints as Qx+c-A^T*lambda'''
   
    Qx = np.matmul(Q, x)
    ATLambda = A.T@mult_eq
    mu = Qx+c-ATLambda
    return mu

In [9]:
def matrix_vector_mult(matrix_, vector_):
    '''Reshape the empty matrix obtained from the multiplication of either an empty matrix or an empty vector'''
    mat_vec = matrix_@vector_
    if matrix_.size == 0 or vector_.size==0:
        mat_vec = mat_vec.reshape(matrix_.shape[0],1)
    return mat_vec

In [10]:
def activesetmethod2(Q, c, A, b, x, L, U, n, k, partition, iter, verbose):
    def log(message):
        if verbose:
            print(message)

    log('It {}'.format(iter))

    U.sort() 
    u = len(U)

    # Problem variables restricted to the set of non-active variables.
    Q_U = Q[np.ix_(U, U)] # Q_UU
    A_U = A[:, U] # Matrix of equality constraints
    x_U = x[U,:] # value of x at the current iteration
    g_U = Q_U@x_U + c[U,:] # gradient of the function at x_U

    # Compute Y and Z  

    # Find the permutation matrix. Step 2 in the pseudocode.
    permutation_m = find_permutation_matrix(U, partition)    
    
    # Rearrange Au s.t AuP = [I|Az] with P permutation matrix
    AuP = A_U@permutation_m 

    # Compute Y and Z as in step 3. 
    Y = csr_matrix(([1]*k, (list(range(k)), list(range(k)))), shape=(u, k))
    Z = compute_Z(AuP, k, u) 

    # Compute py = -h_u = A_ux_u-b. Step 4
    h_U = (A_U@x_U)-b
    py = -h_U

    # Use Cholesky factorization to compute pz. Step 6. 
    # If ZQZ singular status == 0. p will be computed in a different way.
    status, pz = compute_pz(Z, permutation_m, Q_U, Y, py, g_U)

    if status == 1: # Cholesky factorization succeed. pz was computed.

        # Compute pu. Step 7
        Y_py = matrix_vector_mult(Y, py)
        Z_pz = matrix_vector_mult(Z, pz)

        p_U = Y_py + Z_pz
        p_U = permutation_m@p_U
    
        # Compute xu. Step 8
        xu = x_U + p_U 

        if np.all(xu>=-1e-10): # Check feasibility of the new iteration
            
            log('new iter is feasible')
            
            x_ = np.zeros((n,1))
            x_[U,:] = xu

            # Compute Lagrange multipliers lambda. Step  11.
            lag_mult = compute_lambdas(permutation_m, Y, g_U, Q_U, p_U)

            # Compute dual multipliers. Step 12.
            mu = compute_mult_ineq(Q, x_, c, lag_mult, A) 

            # Find the active variables with  with negative dual multiplier.
            args_ = [(v,mu[v]) for v in np.argwhere(mu <= -1e-10)[:,0] if v in set(L)]

            if len(args_) == 0:    # Check optimality.

                L = np.argwhere(abs(x_)<=-1e-10)[:,0]
                U = [i for i in range(len(x)) if i not in L]
                
                log('objective value: {} \n'.format(0.5*x_.T@Q@x_ + x_.T@c))
                
                return x_, L, U, 'the solution is optimal' # Step 14
            
            else: 

                # Select the variable with the most negative dual multiplier. Step 16
                const_to_drop, mu_v = min(args_, key=lambda t: t[1]) 
                # log('mu chosen: {}'.format(mu_v))
                
                # Update the set of active and non-active variables 
                L = L[L != const_to_drop] # Step 17
                U = np.append(U,const_to_drop) # Step 18
                U.sort()

                log('objective value: {}'.format(0.5*x_.T@Q@x_ + x_.T@c) )
                log('var removed from the active set (L) {} \n'.format(const_to_drop))
                
                return x_, L, U, 'the solution is not optimal'
            
        else :

            log('new iter is not feasible')

            # Define the direction p. Step 22
            p_ = np.zeros((n,1))
            p_[U,:] = p_U 

            # Find the step-length parameter. Step 23, 25.
            cand_alpha = [(-x[i,:]/p_[i,:],i) for i in U if p_[i,:] < -1e-10]
            alpha, min_index = min(cand_alpha, key=lambda t: t[0])
            
            # Update the iteration. Step 24
            x = x+alpha*p_

            # Update the sets of active and non-active variables. Steps 26, 27.
            U = U[U != min_index]
            L = np.append(L,min_index)

            log('objective value: {}'.format(0.5*x.T@Q@x + x.T@c))
            log('step chosen: {}'.format(alpha))
            log('var added to the active set: {} \n'.format(min_index))
            
            return x, L, U, 'the solution is not optimal'

    else: # ZQZ singular. 
        log("Cholesky factorization failed")
        
        # Find p with p in Ker(Q_U) st A_Up=0, <c,p>!=0 and in opposite direction of the gradient
        AU_dense = A_U.toarray() 

        AQ_stack = np.vstack((Q_U, AU_dense))

        # Find an orthogonal basis for the null space of Q_U and A_U.
        Z = null_space(AQ_stack)

        # Project c onto Z.
        proj_q = (c[U,:].T @ Z)*Z
        p_U = np.sum(proj_q, axis=1) 
        p_U = p_U.reshape(-1,1)
        
        # Find p in opposite direction to the gradient
        grad = Q_U@x_U+c[U,:]
        proj_grad =np.sum(((A_U@grad)*AU_dense).T / np.sum(np.square(AU_dense), axis=1), axis=1).reshape(-1,1)
        grad = grad - proj_grad 
        p_U = (p_U.T@grad/abs(p_U.T@grad))*p_U

        if np.sign((c[U,:].T@p_U).flatten()[0]) == 1:
            p_U = -p_U

        # Define the direction p for all the variables.
        p_ = np.zeros((n,1))
        p_[U,:] = p_U 
        
        # Find the step-length parameter. Same as step 23.
        cand_alpha = [(-x[i,:]/p_[i,:],i) for i in U if p_[i,:] < -1e-10]
        alpha, min_index = min(cand_alpha, key=lambda t: t[0])

        # Update the iteration
        x = x+alpha*p_

        # Update the set of active and non-active variables.
        U = U[U != min_index]
        L = np.append(L,min_index)

        log('objective value: {}'.format(0.5*x.T@Q@x + x.T@c))
        log('step chosen: {}'.format(alpha))
        log('const added to the active set: {} \n'.format(min_index))
        
        return x, L, U, 'the solution is not optimal'

In [11]:
def active_set_method_optimizer(Q, c, A, b, x0, n, k, verbose, max_iter = 2000):

  # Record the start time for timing the program
  start_time = time.process_time()
  
  if Q.shape[0] != Q.shape[1]:
     raise ValueError('Q must be a square matrix. Receive shape=({},{})'.format(Q.shape[0], Q.shape[1]))
  if c.shape[0] != Q.shape[0]:
     raise ValueError('Q and c must have the same dimension. Received Q as {} and c as {}' % (Q.shape, c.shape))
  if Q.shape[0] != A.shape[1]:
     raise ValueError('Q and A.T must have the same first dimension. Received Q as {} and A as {}' % (Q.shape, A.shape))
  if b.shape[0] != A.shape[0]:
    raise ValueError('The number of rows of A must match the length of b. Received A as {} and b as {}' % (A.shape, b.shape))

  partition = find_the_partition(A)

  # Starting variables
  x = x0
  U = np.nonzero(x)[0]
  L = np.argwhere(x == 0)[:, 0]

  res = 'the solution is not optimal'
  iter = 0

  while res == 'the solution is not optimal' and iter <= max_iter:    
    x, L, U, res = activesetmethod2(Q, c, A, b, x, L, U, n, k, partition, iter, verbose)
    iter +=1

  # Compute the objective value reached
  opt_v = 0.5*x.T@Q@x + x.T@c
  opt_v = opt_v.flatten()[0]

  # Define the final status of the output returned.
  if res == 'the solution is not optimal' and iter > max_iter: 
    status = 'stopped'
  else: 
    status = 'optimal' 

  # Compute the total execution time
  end_time = time.process_time()
  our_algo_time = end_time - start_time

  return opt_v, x, status, iter, our_algo_time


# Experiments

## Functions to run the experiments and print the results

In [12]:
import qpsolvers
from qpsolvers import Problem, solve_problem
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
import scipy.io

In [13]:
#pip install 'qpsolvers[clarabel]'

In [14]:
def clarabel_test_function(G, q, A, dim_m, nconst, verbose=True):
    '''
    P (Union[ndarray, csc_matrix]): Symmetric cost matrix (most solvers require it to be definite as well).
    q (ndarray): Cost vector.
    G (Union[ndarray, csc_matrix, None]) : Linear inequality matrix.
    h (Optional[ndarray]) : Linear inequality vector.
    A (Union[ndarray, csc_matrix, None]) : Linear equality matrix.
    b (Optional[ndarray]) : Linear equality vector.
    lb (Optional[ndarray]) : Lower bound constraint vector. Can contain -np.inf.
    ub (Optional[ndarray]) : Upper bound constraint vector. Can contain +np.inf.
    solver (Optional[str]) : Name of the QP solver, to choose in qpsolvers.available_solvers. This argument is mandatory.
    initvals (Optional[ndarray]) : Primal candidate vector  values used to warm-start the solver.
    verbose (bool) : Set to True to print out extra information.
    '''
    start_time = time.process_time()

    # Set the variables of the problem
    P = G
    q = q
    G_g = np.full((dim_m, dim_m), 0)
    h = np.full((1, dim_m), 0).flatten()
    A = A.toarray()
    b = np.ones((1,nconst)).flatten()
    lb = np.array([0]*dim_m)
    ub = np.array([1]*dim_m)

    # Find the objective value
    x = qpsolvers.solve_qp(P, q, G_g, h, A, b, lb, ub, solver='clarabel')
    problem = Problem(P, q, G_g, h, A, b)
    solution = solve_problem(problem, 'clarabel')
    
    # Compute the total execution time
    end_time = time.process_time()
    gpu_time = end_time- start_time

    # Extract additional info of the solution
    extras = solution.extras
    x_extr = solution.x
    found = solution.found

    return x, x_extr, found, solution, gpu_time, extras



In [15]:
def gen_variables_for_tests_except_Q(Q, dim_m, ncons, c_type):

    # Generate the partition that defines the equality constraints
    part = partition(dim_m, ncons)

    # Compute the matrix of constraints A
    A = matrix_of_constraints(part)

    # Define the vector b s.t Ax=b
    b = define_vector_b(ncons)

    # Compute the initial point and the sets of active and non-active constraints
    x0 = initial_point(part)

    # Define the linear part of the problem
    if c_type=='basic':
        c = generate_c_vector(dim_m)
    elif c_type=='special':
        c = generate_special_c_vector(Q,x0)
        
    return c, A, b, x0 

In [16]:
import quadprog

def quadprog_testing_function(G, q, A, b, dim_m, nconst):
    start_time = time.process_time()
    A = np.vstack((A.toarray(), np.identity(dim_m)))
    b = np.vstack((b, np.zeros((dim_m,1))))
    solution = quadprog.solve_qp(G, -q.reshape(-1), A.T, b.reshape(-1), meq=nconst)
    x = solution[0]
    opt_v = solution[1]
    iterations = solution[3]
    end_time = time.process_time()
    quadprog_time = end_time - start_time
    
    return x, opt_v, quadprog_time, iterations

In [17]:
def run_exp_and_populate_dict(Q, c, A, b, x0, dim_m, ncons, solver='clarabel', verbose=False, max_iter=2000):
        
    result = {'n': dim_m, 'k': ncons}
    
    # Run asqp
    opt_v, x_asqp, status, iter, asqp_time = active_set_method_optimizer(Q, c, A, b, x0, dim_m, ncons, verbose, max_iter)
    
    result['asqp'] = {"obj_v": opt_v, "x": x_asqp, "status": status, "time": round(asqp_time,4)}
    
    if solver=='clarabel':
        # Run Clarabel
        x, x_extr, found, solution, time_c, extra = clarabel_test_function(Q, c, A, dim_m, ncons)
        status_c = extra['status']
        
        if found==True and x is None:
            x=x_extr
        if found==True:
            obj_v = 0.5*x.T@Q@x + c.T@x
        elif found==False and  x is not None:
            obj_v = 0.5*x.T@Q@x + c.T@x   
        else:
            obj_v = None
        
        result['clarabel'] = {"obj_v": obj_v, "x": x, "status": status_c, "time": time_c, "extra": extra}
    else:
        # Run quadprog
        x_quad, obj_v, time_quadp, iterations = quadprog_testing_function(Q, c, A, b, dim_m, ncons)

        result['quadprog'] = {"obj_v": obj_v, "x": x_quad, "time": time_quadp, "quad_iterations":iterations, "status": 'NA'}

    return result

In [18]:
def print_results(results, solver='clarabel'):

    table_data = []
    
    for res in results:
        if res[solver]['obj_v'] is not None:
            asqp_v = res['asqp']['obj_v']
            solver_v = res[solver]['obj_v']
            # Relative gap
            ref = min(asqp_v, solver_v)
            oth_ = max(asqp_v, solver_v)
            rel_gap = (oth_ - ref)/max(1,abs(ref))

            # Absolute gap
            abs_gap = asqp_v - solver_v
        else:
            rel_gap = None
            abs_gap = None

        table_data.append([res["k"], 
                           res["n"], 
                           res['asqp']['status'], 
                           res['asqp']['obj_v'], 
                           res[solver]['obj_v'], 
                           res['asqp']['time'], 
                           res[solver]['time'],
                           abs_gap, 
                           rel_gap])
        
    headers = ["k", 
               "n",
               "ASQPstat",
               "asqp_sol",
               solver+"_sol",
               "asqp_time",
               solver+"_time",
               "abs_gap",
               "rel_gap"]
    
    print(tabulate(table_data, headers=headers, tablefmt="github"))


In [19]:
sp_sizes = [100, 200, 500, 1000]
k_values = [10, 50, 90]

## 1,2 Q positive semidefinite, full rank, dense matrix

### Basic case: q random

In [20]:
exp_numb = 1
results_1 = []

for k in k_values:
    for n in sp_sizes:
        print('K =:',str(k),'Size = ',str(n))
        Q = generate_symmetric_psd_matrix(n,n)
        c, A, b, x0 = gen_variables_for_tests_except_Q(Q, n, k, 'basic')
        result = run_exp_and_populate_dict(Q, c, A, b, x0, n, k)
        results_1.append(result)

K =: 10 Size =  100
K =: 10 Size =  200
K =: 10 Size =  500
K =: 10 Size =  1000
K =: 50 Size =  100
K =: 50 Size =  200
K =: 50 Size =  500
K =: 50 Size =  1000
K =: 90 Size =  100
K =: 90 Size =  200
K =: 90 Size =  500
K =: 90 Size =  1000


In [21]:
print_results(results_1)

|   k |    n | ASQPstat   |         asqp_sol |     clarabel_sol |   asqp_time |   clarabel_time |      abs_gap |     rel_gap |
|-----|------|------------|------------------|------------------|-------------|-----------------|--------------|-------------|
|  10 |  100 | optimal    |   1598.53        |   1598.53        |      0.4219 |        0.015625 | -6.01344e-08 | 3.76186e-11 |
|  10 |  200 | optimal    |   5471.05        |   5471.05        |      0.625  |        0.109375 | -1.31948e-06 | 2.41175e-10 |
|  10 |  500 | optimal    |  25747.9         |  25747.9         |      4.1875 |        0.71875  | -1.35697e-05 | 5.27022e-10 |
|  10 | 1000 | optimal    |  45951.8         |  45951.8         |     24.875  |        5.34375  | -1.60404e-05 | 3.49069e-10 |
|  50 |  100 | optimal    | 115974           | 115974           |      0.2031 |        0.03125  | -4.37837e-05 | 3.7753e-10  |
|  50 |  200 | optimal    | 162370           | 162370           |      0.7188 |        0.078125 | -5.84064e-05 

###  Special case c = -Q*x0

In [22]:
exp_numb = 2
results_2 = []

for k in k_values:
    for n in sp_sizes:
        print('K =:',str(k),'Size = ',str(n))
        Q = generate_symmetric_psd_matrix(n,n)
        c, A, b, x0 = gen_variables_for_tests_except_Q(Q, n, k, 'special')
        result = run_exp_and_populate_dict(Q, c, A, b, x0, n, k)
        results_2.append(result)

K =: 10 Size =  100
K =: 10 Size =  200
K =: 10 Size =  500
K =: 10 Size =  1000
K =: 50 Size =  100
K =: 50 Size =  200
K =: 50 Size =  500
K =: 50 Size =  1000
K =: 90 Size =  100
K =: 90 Size =  200
K =: 90 Size =  500
K =: 90 Size =  1000


In [23]:
print_results(results_2)

|   k |    n | ASQPstat   |          asqp_sol |      clarabel_sol |   asqp_time |   clarabel_time |      abs_gap |     rel_gap |
|-----|------|------------|-------------------|-------------------|-------------|-----------------|--------------|-------------|
|  10 |  100 | optimal    |  -52181.3         |  -52181.3         |      0.0156 |        0        | -9.05091e-05 | 1.73451e-09 |
|  10 |  200 | optimal    | -161663           | -161663           |      0      |        0.125    | -0.000123729 | 7.65349e-10 |
|  10 |  500 | optimal    |      -1.26365e+06 |      -1.26365e+06 |      0.0156 |        0.8125   | -0.00252417  | 1.99753e-09 |
|  10 | 1000 | optimal    |      -5.30861e+06 |      -5.30861e+06 |      0.0469 |        5.70312  | -0.0150221   | 2.82976e-09 |
|  50 |  100 | optimal    | -272382           | -272382           |      0.0156 |        0.046875 | -0.000222733 | 8.17723e-10 |
|  50 |  200 | optimal    | -955715           | -955715           |      0      |        0.140625

## 3,4 Q positive semidefinite, low rank, dense matrix

### Basic case

In [24]:
exp_numb = 3
results_3 = []

for k in k_values:
    for n in sp_sizes:
        print('K =:',str(k),'Size = ',str(n))
        Q = generate_symmetric_psd_matrix(n,50)
        c, A, b, x0 = gen_variables_for_tests_except_Q(Q, n, k, 'basic')

        max_iter = 2000
        if n>=500:
            max_iter = 5000

        result = run_exp_and_populate_dict(Q, c, A, b, x0, n, k, 'clarabel', verbose=False, max_iter=max_iter)  
        results_3.append(result)

K =: 10 Size =  100
K =: 10 Size =  200
K =: 10 Size =  500
K =: 10 Size =  1000
K =: 50 Size =  100
K =: 50 Size =  200
K =: 50 Size =  500
K =: 50 Size =  1000
K =: 90 Size =  100
K =: 90 Size =  200
K =: 90 Size =  500
K =: 90 Size =  1000


In [25]:
print_results(results_3)

|   k |    n | ASQPstat   |     asqp_sol |   clarabel_sol |   asqp_time |   clarabel_time |      abs_gap |     rel_gap |
|-----|------|------------|--------------|----------------|-------------|-----------------|--------------|-------------|
|  10 |  100 | optimal    |    502.383   |      502.383   |      0.25   |        0.03125  | -1.98366e-06 | 3.9485e-09  |
|  10 |  200 | optimal    |    203.497   |      203.497   |      0.5781 |        0.09375  | -3.25304e-08 | 1.59857e-10 |
|  10 |  500 | optimal    |      1.34422 |        1.34422 |      4.6719 |        0.828125 | -3.46435e-10 | 2.57723e-10 |
|  10 | 1000 | optimal    |      1.00704 |        1.00704 |     14.5625 |        6.57812  | -7.07692e-10 | 7.02745e-10 |
|  50 |  100 | optimal    |  39860.3     |    39860.3     |      0.2812 |        0.03125  | -1.41652e-05 | 3.5537e-10  |
|  50 |  200 | optimal    |   7081.03    |     7081.03    |      0.5625 |        0.125    | -2.85725e-05 | 4.03508e-09 |
|  50 |  500 | optimal    |     

### Special case

In [26]:
exp_numb = 4
results_4 = []
for k in k_values:
    for n in sp_sizes:
        print('K =:',str(k),'Size = ',str(n))
        Q = generate_symmetric_psd_matrix(n,50)
        c, A, b, x0 = gen_variables_for_tests_except_Q(Q, n, k, 'special')

        max_iter = 2000
        if n>=500:
            max_iter = 5000

        result = run_exp_and_populate_dict(Q, c, A, b, x0, n, k, 'clarabel', verbose=False, max_iter=max_iter)   
        results_4.append(result)

K =: 10 Size =  100
K =: 10 Size =  200
K =: 10 Size =  500
K =: 10 Size =  1000
K =: 50 Size =  100
K =: 50 Size =  200
K =: 50 Size =  500
K =: 50 Size =  1000
K =: 90 Size =  100
K =: 90 Size =  200
K =: 90 Size =  500
K =: 90 Size =  1000


In [27]:
print_results(results_4)

|   k |    n | ASQPstat   |          asqp_sol |   clarabel_sol |   asqp_time |   clarabel_time |      abs_gap |     rel_gap |
|-----|------|------------|-------------------|----------------|-------------|-----------------|--------------|-------------|
|  10 |  100 | optimal    |  -27899.8         |       -27899.8 |      0      |        0.03125  | -3.66729e-05 | 1.31445e-09 |
|  10 |  200 | optimal    |  -37705.1         |       -37705.1 |      0.0156 |        0.125    | -7.44737e-05 | 1.97516e-09 |
|  10 |  500 | optimal    | -145090           |      -145090   |      0.0312 |        0.828125 | -0.000128322 | 8.84432e-10 |
|  10 | 1000 | optimal    | -291859           |      -291859   |      0.0312 |        5.76562  | -0.000836412 | 2.86581e-09 |
|  50 |  100 | optimal    | -116578           |      -116578   |      0      |        0.03125  | -0.000100625 | 8.63159e-10 |
|  50 |  200 | optimal    | -298674           |      -298674   |      0.0156 |        0.109375 | -0.000420692 | 1.4085

## 5,6 Q positive semidefinite, full rank, sparse matrix

### Basic case

In [28]:
exp_numb = 5
results_5 = []
for k in k_values:
    for i in sp_sizes:
        print('K =:',str(k),'Size = ',str(n))
        mat = scipy.io.loadmat('C:/Users/SteffaniaSierraG/Documents/ODS_PROJECT/active_set_method_code/ActiveSetAlgorithm/SparseMatrices/matrix'+str(n)+'spar0.2.mat')
        sparse_matlab = mat['sparse_psd']
        sparse_python = csr_matrix(sparse_matlab)
        Q = sparse_python.toarray()
        c, A, b, x0 = gen_variables_for_tests_except_Q(Q, n, k, 'basic')
        result = run_exp_and_populate_dict(Q, c, A, b, x0, n, k)  
        results_5.append(result)     

K =: 10 Size =  1000
K =: 10 Size =  1000
K =: 10 Size =  1000
K =: 10 Size =  1000
K =: 50 Size =  1000
K =: 50 Size =  1000
K =: 50 Size =  1000
K =: 50 Size =  1000
K =: 90 Size =  1000
K =: 90 Size =  1000
K =: 90 Size =  1000
K =: 90 Size =  1000


In [29]:
print_results(results_5)

|   k |    n | ASQPstat   |   asqp_sol |   clarabel_sol |   asqp_time |   clarabel_time |      abs_gap |     rel_gap |
|-----|------|------------|------------|----------------|-------------|-----------------|--------------|-------------|
|  10 | 1000 | optimal    |   0.550542 |       0.550542 |      1.3438 |         1.21875 | -7.13104e-10 | 7.13104e-10 |
|  10 | 1000 | optimal    |   0.631969 |       0.631969 |      2.4219 |         1.23438 | -2.77227e-10 | 2.77227e-10 |
|  10 | 1000 | optimal    |   0.738325 |       0.738325 |      2.1562 |         1.17188 | -3.29783e-10 | 3.29783e-10 |
|  10 | 1000 | optimal    |   0.669249 |       0.669249 |      1.4688 |         1.15625 | -9.32223e-10 | 9.32223e-10 |
|  50 | 1000 | optimal    |  11.2982   |      11.2982   |      5.5469 |         1.14062 | -1.4044e-09  | 1.24303e-10 |
|  50 | 1000 | optimal    |   9.66496  |       9.66496  |      5.5625 |         1.20312 | -1.57086e-08 | 1.62531e-09 |
|  50 | 1000 | optimal    |  12.8338   |      12

#### Special case

In [30]:
exp_numb = 6
results_6 = []
for k in k_values:
    for n in sp_sizes:
        print('K =:',str(k),'Size = ',str(n))
        mat = scipy.io.loadmat('C:/Users/SteffaniaSierraG/Documents/ODS_PROJECT/active_set_method_code/ActiveSetAlgorithm/SparseMatrices/matrix'+str(n)+'spar0.2.mat')
        sparse_matlab = mat['sparse_psd']
        sparse_python = csr_matrix(sparse_matlab)
        Q = sparse_python.toarray()
        c, A, b, x0 = gen_variables_for_tests_except_Q(Q, n, k, 'special')
        result = run_exp_and_populate_dict(Q, c, A, b, x0, n, k)
        results_6.append(result)
            

K =: 10 Size =  100
K =: 10 Size =  200
K =: 10 Size =  500
K =: 10 Size =  1000
K =: 50 Size =  100
K =: 50 Size =  200
K =: 50 Size =  500
K =: 50 Size =  1000
K =: 90 Size =  100
K =: 90 Size =  200
K =: 90 Size =  500
K =: 90 Size =  1000


In [31]:
print_results(results_6)

|   k |    n | ASQPstat   |   asqp_sol |   clarabel_sol |   asqp_time |   clarabel_time |      abs_gap |     rel_gap |
|-----|------|------------|------------|----------------|-------------|-----------------|--------------|-------------|
|  10 |  100 | optimal    |  -2.08033  |      -2.08033  |      0      |        0.015625 | -6.2881e-09  | 3.02265e-09 |
|  10 |  200 | optimal    |  -1.98645  |      -1.98645  |      0      |        0.0625   | -1.80342e-09 | 9.07861e-10 |
|  10 |  500 | optimal    |  -0.788304 |      -0.788304 |      0      |        0.3125   | -4.01242e-10 | 4.01242e-10 |
|  10 | 1000 | optimal    |  -3.33532  |      -3.33532  |      0.0156 |        1.54688  | -4.13527e-09 | 1.23984e-09 |
|  50 |  100 | optimal    | -10.3473   |     -10.3473   |      0      |        0.03125  | -1.08956e-08 | 1.05299e-09 |
|  50 |  200 | optimal    | -14.8684   |     -14.8684   |      0      |        0.0625   | -3.52583e-08 | 2.37135e-09 |
|  50 |  500 | optimal    |  -6.08087  |      -6

##  7,8 Q positive semidefinite, full rank, dense matrix

#### Basic

In [32]:
exp_numb = 7
results_7 = []
for k in k_values:
    for n in sp_sizes:
        print(f"K =: {k} Size = {n}")
        Q = generate_symmetric_pd_matrix(n)
        c, A, b, x0 = gen_variables_for_tests_except_Q(Q, n, k, 'basic')
        result = run_exp_and_populate_dict(Q, c, A, b, x0, n, k, 'quadprog')
        results_7.append(result)

K =: 10 Size = 100
K =: 10 Size = 200
K =: 10 Size = 500
K =: 10 Size = 1000
K =: 50 Size = 100
K =: 50 Size = 200
K =: 50 Size = 500
K =: 50 Size = 1000
K =: 90 Size = 100
K =: 90 Size = 200
K =: 90 Size = 500
K =: 90 Size = 1000


In [33]:
print_results(results_7, 'quadprog') 

|   k |    n | ASQPstat   |   asqp_sol |   quadprog_sol |   asqp_time |   quadprog_time |      abs_gap |     rel_gap |
|-----|------|------------|------------|----------------|-------------|-----------------|--------------|-------------|
|  10 |  100 | optimal    |    1078.93 |        1078.93 |      0.1094 |        0        | -5.6616e-11  | 5.24744e-14 |
|  10 |  200 | optimal    |    2258.68 |        2258.68 |      0.1562 |        0.0625   | -4.2337e-10  | 1.87441e-13 |
|  10 |  500 | optimal    |    5834.14 |        5834.14 |      0.4062 |        0.671875 |  3.75621e-10 | 6.43834e-14 |
|  10 | 1000 | optimal    |   11693.3  |       11693.3  |      1.2344 |        4.76562  |  5.18412e-10 | 4.43339e-14 |
|  50 |  100 | optimal    |   29607.3  |       29607.3  |      0.1406 |        0.03125  | -2.58296e-10 | 8.72409e-15 |
|  50 |  200 | optimal    |   59334.4  |       59334.4  |      0.4219 |        0.0625   | -6.1118e-10  | 1.03006e-14 |
|  50 |  500 | optimal    |  148076    |      14

#### Special

In [34]:
exp_numb = 8
results_8 = []
for k in k_values:
    for n in sp_sizes:
        print(f"K =: {k} Size = {n}")
        Q = generate_symmetric_pd_matrix(n)
        c, A, b, x0 = gen_variables_for_tests_except_Q(Q, n, k, 'special')
        result = run_exp_and_populate_dict(Q, c, A, b, x0, n, k, 'quadprog')
        results_8.append(result)

K =: 10 Size = 100
K =: 10 Size = 200
K =: 10 Size = 500
K =: 10 Size = 1000
K =: 50 Size = 100
K =: 50 Size = 200
K =: 50 Size = 500
K =: 50 Size = 1000
K =: 90 Size = 100
K =: 90 Size = 200
K =: 90 Size = 500
K =: 90 Size = 1000


In [35]:
print_results(results_8, 'quadprog') 

|   k |    n | ASQPstat   |          asqp_sol |      quadprog_sol |   asqp_time |   quadprog_time |      abs_gap |     rel_gap |
|-----|------|------------|-------------------|-------------------|-------------|-----------------|--------------|-------------|
|  10 |  100 | optimal    |   -1369.71        |   -1369.71        |      0      |        0        |  2.27374e-13 | 1.66002e-16 |
|  10 |  200 | optimal    |   -2614.06        |   -2614.06        |      0.0156 |        0.046875 |  9.09495e-13 | 3.47925e-16 |
|  10 |  500 | optimal    |   -6598.84        |   -6598.84        |      0.0312 |        0.671875 | -4.54747e-12 | 6.89132e-16 |
|  10 | 1000 | optimal    |  -12716.4         |  -12716.4         |      0.0312 |        4.64062  |  2.00089e-11 | 1.57347e-15 |
|  50 |  100 | optimal    |  -32007           |  -32007           |      0      |        0.03125  | -3.63798e-12 | 1.13662e-16 |
|  50 |  200 | optimal    |  -62092.7         |  -62092.7         |      0      |        0.0625  